# Processamento de Dados de Vendas (2022-2024)
Este notebook é dedicado ao tratamento abrangente de dados de vendas abrangendo o período de 2022 a 2024. Sua função principal é padronizar e corrigir os dados brutos de acordo com as regras comerciais estabelecidas, garantindo a integridade e a confiabilidade do conjunto de dados.

In [2]:
# Defininindo o arquivo de trabalho

arquivo = '../data/raw/vendas_22-24.csv'

In [4]:
# Importação das bibliotecas utilizadas.

import pandas as pd
import statistics as sts

In [ ]:
# Carregamento do arquivo de dados a ser utilizado.

dados = pd.read_csv(arquivo, delimiter=';')
dados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Data             10000 non-null  object 
 1   Loja             9672 non-null   object 
 2   Produto          10000 non-null  object 
 3   Categoria        10000 non-null  object 
 4   Quantidade       9691 non-null   float64
 5   Preço Unitário   9688 non-null   float64
 6   Total Venda      7979 non-null   float64
 7   Cliente_ID       9732 non-null   float64
 8   Forma_Pagamento  9684 non-null   object 
dtypes: float64(4), object(5)
memory usage: 703.3+ KB
